In [1]:
!pip install --upgrade mysql-connector-python


[notice] A new release of pip is available: 25.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np
import  pymysql

In [3]:
providers_data= pd.read_csv('providers_data.csv')
recivers_data= pd.read_csv('receivers_data.csv')
foodlisting_data= pd.read_csv('food_listings_data.csv')
claims_data= pd.read_csv('claims_data.csv')

In [4]:
providers_data

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577
...,...,...,...,...,...,...
995,996,"Vasquez, Ruiz and Flowers",Restaurant,"84308 Justin Stravenue\nNew Amberside, NE 53447",Williamview,+1-319-378-7627x0682
996,997,Garza-Williams,Catering Service,"08864 Figueroa Radial Suite 948\nJennaberg, AZ...",East Rossside,001-924-441-3963x746
997,998,Novak Group,Grocery Store,"934 Zachary Run\nMelissamouth, WY 02729",Joshuastad,(903)642-1969x3300
998,999,Moody Ltd,Grocery Store,"17580 Ernest Hills\nLake Michaelmouth, OR 56416",Stevenchester,637.300.3664x4880


In [5]:
providers_data.isna().sum()

Provider_ID    0
Name           0
Type           0
Address        0
City           0
Contact        0
dtype: int64

In [6]:
providers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Provider_ID  1000 non-null   int64 
 1   Name         1000 non-null   object
 2   Type         1000 non-null   object
 3   Address      1000 non-null   object
 4   City         1000 non-null   object
 5   Contact      1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


In [7]:
recivers_data

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856
3,4,Erika Rose,NGO,South Shaneville,8296491111
4,5,John Romero,Individual,Bakerport,067.491.0154
...,...,...,...,...,...
995,996,Matthew Curtis,Shelter,Lanechester,127-889-4442x1289
996,997,Amanda Cain,NGO,New Steven,+1-001-491-5601x5316
997,998,Theodore Briggs,Individual,South Sandra,930-609-9442x5031
998,999,Cheyenne Ramsey,NGO,Lake Jeffery,001-326-320-4816x15300


In [8]:
recivers_data.isna().sum()

Receiver_ID    0
Name           0
Type           0
City           0
Contact        0
dtype: int64

In [9]:
recivers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Receiver_ID  1000 non-null   int64 
 1   Name         1000 non-null   object
 2   Type         1000 non-null   object
 3   City         1000 non-null   object
 4   Contact      1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


In [10]:
foodlisting_data

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,3/28/2025,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,3/16/2025,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,3/19/2025,279,Restaurant,Garciaport,Vegan,Dinner
...,...,...,...,...,...,...,...,...,...
995,996,Fish,15,3/30/2025,467,Catering Service,Phillipsfort,Vegan,Breakfast
996,997,Fish,22,3/18/2025,35,Grocery Store,Andersonmouth,Vegetarian,Breakfast
997,998,Fruits,6,3/22/2025,444,Restaurant,New Billy,Non-Vegetarian,Dinner
998,999,Pasta,15,3/30/2025,702,Supermarket,Lake Mistyton,Non-Vegetarian,Lunch


In [11]:
foodlisting_data.isna().sum()

Food_ID          0
Food_Name        0
Quantity         0
Expiry_Date      0
Provider_ID      0
Provider_Type    0
Location         0
Food_Type        0
Meal_Type        0
dtype: int64

In [12]:
foodlisting_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Food_ID        1000 non-null   int64 
 1   Food_Name      1000 non-null   object
 2   Quantity       1000 non-null   int64 
 3   Expiry_Date    1000 non-null   object
 4   Provider_ID    1000 non-null   int64 
 5   Provider_Type  1000 non-null   object
 6   Location       1000 non-null   object
 7   Food_Type      1000 non-null   object
 8   Meal_Type      1000 non-null   object
dtypes: int64(3), object(6)
memory usage: 70.4+ KB


In [13]:
foodlisting_data['Expiry_Date']= pd.to_datetime(foodlisting_data['Expiry_Date'])
foodlisting_data

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,2025-03-28,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,2025-03-16,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,2025-03-19,279,Restaurant,Garciaport,Vegan,Dinner
...,...,...,...,...,...,...,...,...,...
995,996,Fish,15,2025-03-30,467,Catering Service,Phillipsfort,Vegan,Breakfast
996,997,Fish,22,2025-03-18,35,Grocery Store,Andersonmouth,Vegetarian,Breakfast
997,998,Fruits,6,2025-03-22,444,Restaurant,New Billy,Non-Vegetarian,Dinner
998,999,Pasta,15,2025-03-30,702,Supermarket,Lake Mistyton,Non-Vegetarian,Lunch


In [14]:
foodlisting_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Food_ID        1000 non-null   int64         
 1   Food_Name      1000 non-null   object        
 2   Quantity       1000 non-null   int64         
 3   Expiry_Date    1000 non-null   datetime64[ns]
 4   Provider_ID    1000 non-null   int64         
 5   Provider_Type  1000 non-null   object        
 6   Location       1000 non-null   object        
 7   Food_Type      1000 non-null   object        
 8   Meal_Type      1000 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 70.4+ KB


In [15]:
claims_data

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24
2,3,626,492,Completed,3/21/2025 0:59
3,4,61,933,Cancelled,3/4/2025 9:08
4,5,345,229,Pending,3/14/2025 15:17
...,...,...,...,...,...
995,996,855,211,Completed,3/13/2025 19:40
996,997,980,746,Cancelled,3/17/2025 22:04
997,998,832,967,Cancelled,3/13/2025 18:00
998,999,917,90,Completed,3/1/2025 15:31


In [16]:
claims_data.isna().sum()

Claim_ID       0
Food_ID        0
Receiver_ID    0
Status         0
Timestamp      0
dtype: int64

In [17]:
claims_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Claim_ID     1000 non-null   int64 
 1   Food_ID      1000 non-null   int64 
 2   Receiver_ID  1000 non-null   int64 
 3   Status       1000 non-null   object
 4   Timestamp    1000 non-null   object
dtypes: int64(3), object(2)
memory usage: 39.2+ KB


In [18]:
claims_data['Timestamp']=pd.to_datetime(claims_data['Timestamp'])
claims_data

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,2025-03-05 05:26:00
1,2,353,391,Cancelled,2025-03-11 10:24:00
2,3,626,492,Completed,2025-03-21 00:59:00
3,4,61,933,Cancelled,2025-03-04 09:08:00
4,5,345,229,Pending,2025-03-14 15:17:00
...,...,...,...,...,...
995,996,855,211,Completed,2025-03-13 19:40:00
996,997,980,746,Cancelled,2025-03-17 22:04:00
997,998,832,967,Cancelled,2025-03-13 18:00:00
998,999,917,90,Completed,2025-03-01 15:31:00


In [19]:
claims_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Claim_ID     1000 non-null   int64         
 1   Food_ID      1000 non-null   int64         
 2   Receiver_ID  1000 non-null   int64         
 3   Status       1000 non-null   object        
 4   Timestamp    1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 39.2+ KB


In [20]:
con= pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    autocommit=True
    )
print(con)

In [21]:
mycursor=con.cursor()

In [22]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS LOCALFOODWASTE")

1

In [23]:
mycursor.execute("SHOW DATABASES")
for x in mycursor:
    print(x)

('hari',)
('information_schema',)
('localfoodwaste',)
('mysql',)
('performance_schema',)
('videogame',)
('videogames_db',)


In [24]:
mycursor.execute("USE localfoodwaste")

0

In [25]:
create_providers_table = """
CREATE TABLE IF NOT EXISTS Providers (
    Provider_ID INT PRIMARY KEY,
    Name VARCHAR(255) NOT NULL,
    Type VARCHAR(50),
    Address TEXT,
    City VARCHAR(100),
    Contact VARCHAR(100)
);
"""

create_receivers_table = """
CREATE TABLE IF NOT EXISTS Receivers (
    Receiver_ID INT PRIMARY KEY,
    Name VARCHAR(255) NOT NULL,
    Type VARCHAR(50),
    City VARCHAR(100),
    Contact VARCHAR(100)
);
"""

create_foodlistings_table = """
CREATE TABLE IF NOT EXISTS FoodListings (
    Food_ID INT PRIMARY KEY,
    Food_Name VARCHAR(100),
    Quantity INT,
    Expiry_Date DATE,
    Provider_ID INT,
    Provider_Type VARCHAR(50),
    Location VARCHAR(100),
    Food_Type VARCHAR(50),
    Meal_Type VARCHAR(50),
    FOREIGN KEY (Provider_ID) REFERENCES Providers(Provider_ID)
    ON DELETE CASCADE
    ON UPDATE CASCADE
);
"""

create_claims_table = """
CREATE TABLE IF NOT EXISTS Claims (
    Claim_ID INT PRIMARY KEY,
    Food_ID INT,
    Receiver_ID INT,
    Status VARCHAR(50),
    Timestamp DATETIME,
    FOREIGN KEY (Food_ID) REFERENCES FoodListings(Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES Receivers(Receiver_ID)
    ON DELETE CASCADE
    ON UPDATE CASCADE
);
"""
mycursor.execute(create_providers_table)
mycursor.execute(create_receivers_table)
print("Providers and Receivers tables created.")
mycursor.execute(create_foodlistings_table)
mycursor.execute(create_claims_table)
print("FoodListings and Claims tables created.")

Providers and Receivers tables created.
FoodListings and Claims tables created.


In [26]:
mycursor.execute("show tables") #will return list of tuples in single row for each table
tables = mycursor.fetchall()
for table in tables:
    print(table[0])

claims
foodlistings
providers
receivers


In [27]:
insert_query_providers = """
INSERT INTO Providers (Provider_ID, Name, Type, Address, City, Contact) 
VALUES (%s, %s, %s, %s, %s, %s)
"""
mycursor.execute("SELECT COUNT(*) FROM Providers")
count = mycursor.fetchone()[0]
if count == 0:
  for index, row in providers_data.iterrows():
    mycursor.execute(insert_query_providers, tuple(row))
  print(" Providers data inserted.")
else:
  print("Data already present")

 Providers data inserted.


In [28]:
select_providers = "SELECT * FROM Providers LIMIT 5"
mycursor.execute(select_providers)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)


   Provider_ID                         Name           Type  \
0            1             Gonzales-Cochran    Supermarket   
1            2  Nielsen, Johnson and Fuller  Grocery Store   
2            3                 Miller-Black    Supermarket   
3            4   Clark, Prince and Williams  Grocery Store   
4            5               Coleman-Farley  Grocery Store   

                                             Address            City  \
0  74347 Christopher Extensions\nAndreamouth, OK ...     New Jessica   
1           91228 Hanson Stream\nWelchtown, OR 27136     East Sheena   
2  561 Martinez Point Suite 507\nGuzmanchester, W...  Lake Jesusview   
3     467 Bell Trail Suite 409\nPort Jesus, IA 61188     Mendezmouth   
4  078 Matthew Creek Apt. 319\nSaraborough, MA 53978   Valentineside   

                Contact  
0       +1-600-220-0480  
1  +1-925-283-8901x6297  
2      001-517-295-2206  
3      556.944.8935x401  
4          193.714.6577  


In [30]:
mycursor.execute("SELECT COUNT(*) FROM Receivers")
count = mycursor.fetchone()[0]

if count == 0:
    insert_query_receivers = """
    INSERT INTO Receivers (Receiver_ID, Name, Type, City, Contact) 
    VALUES (%s, %s, %s, %s, %s)
    """
    for index, row in recivers_data.iterrows():
        mycursor.execute(insert_query_receivers, tuple(row))
    print("Receivers data inserted.")
else:
    print("Receivers table already has data. Skipping insert.")

Receivers table already has data. Skipping insert.


In [31]:
select_recivers="select * from  Receivers LIMIT 5"
mycursor.execute(select_recivers)
rows=mycursor.fetchall()
column_recivers= [ desc[0] for desc in mycursor.description]
recivers_df= pd.DataFrame(rows,columns=column_recivers)
print(recivers_df)


   Receiver_ID          Name        Type               City           Contact
0            1  Donald Gomez     Shelter     Port Carlburgh     (955)922-5295
1            2  Laurie Ramos  Individual         Lewisburgh      761.042.1570
2            3  Ashley Mckee         NGO  South Randalltown  691-023-0094x856
3            4    Erika Rose         NGO   South Shaneville        8296491111
4            5   John Romero  Individual          Bakerport      067.491.0154


In [33]:
mycursor.execute("SELECT COUNT(*) FROM FoodListings")
count = mycursor.fetchone()[0]

if count == 0:
    insert_query_food = """
    INSERT INTO FoodListings (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    for index, row in foodlisting_data.iterrows():
        mycursor.execute(insert_query_food, tuple(row))
    print("FoodListings data inserted.")
else:
    print("FoodListings table already has data. Skipping insert.")

FoodListings table already has data. Skipping insert.


In [34]:
select_foodlisting="select * from  FoodListings LIMIT 5"
mycursor.execute(select_foodlisting)
rows=mycursor.fetchall()
column_foodlisting= [ desc[0] for desc in mycursor.description]
foodlisting_df= pd.DataFrame(rows,columns=column_foodlisting)
print(foodlisting_df)


   Food_ID Food_Name  Quantity Expiry_Date  Provider_ID     Provider_Type  \
0        1     Bread        43  2025-03-17          110     Grocery Store   
1        2      Soup        22  2025-03-24          791     Grocery Store   
2        3    Fruits        46  2025-03-28          478  Catering Service   
3        4    Fruits        15  2025-03-16          930        Restaurant   
4        5      Soup        14  2025-03-19          279        Restaurant   

           Location       Food_Type  Meal_Type  
0  South Kellyville  Non-Vegetarian  Breakfast  
1        West James  Non-Vegetarian     Dinner  
2       Lake Regina           Vegan  Breakfast  
3         Kellytown           Vegan      Lunch  
4        Garciaport           Vegan     Dinner  


In [36]:
mycursor.execute("SELECT COUNT(*) FROM Claims")
count = mycursor.fetchone()[0]

if count == 0:
    insert_query_claims = """
    INSERT INTO Claims (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp) 
    VALUES (%s, %s, %s, %s, %s)
    """
    for index, row in claims_data.iterrows():
        mycursor.execute(insert_query_claims, tuple(row))
    print("Claims data inserted.")
else:
    print("Claims table already has data. Skipping insert.")

Claims table already has data. Skipping insert.


In [37]:
select_claims="select * from Claims LIMIT 5"
mycursor.execute(select_claims)
rows=mycursor.fetchall()
column_claims=[desc[0] for desc in mycursor.description]
claims_df=pd.DataFrame(rows,columns=column_claims)
print(claims_df)

   Claim_ID  Food_ID  Receiver_ID     Status           Timestamp
0         1      164          908    Pending 2025-03-05 05:26:00
1         2      353          391  Cancelled 2025-03-11 10:24:00
2         3      626          492  Completed 2025-03-21 00:59:00
3         4       61          933  Cancelled 2025-03-04 09:08:00
4         5      345          229    Pending 2025-03-14 15:17:00


In [38]:
#1.Questions
query1 = """
        SELECT 
            City,
            COUNT(DISTINCT Provider_ID) AS Total_Providers,
            COUNT(DISTINCT Receiver_ID) AS Total_Receivers
        FROM (
            SELECT City, Provider_ID, NULL AS Receiver_ID FROM Providers
            UNION ALL
            SELECT City, NULL AS Provider_ID, Receiver_ID FROM Receivers
        ) AS combined
        GROUP BY City;
        """

mycursor.execute(query1)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

                City  Total_Providers  Total_Receivers
0         Aaronshire                0                1
1           Adambury                1                0
2           Adamland                0                1
3          Adamsview                1                0
4         Adamsville                1                0
...              ...              ...              ...
1873       Yatesside                1                0
1874    Youngchester                1                0
1875     Zacharyview                0                1
1876    Zimmermanton                1                0
1877  Zimmermanville                1                0

[1878 rows x 3 columns]


In [39]:
#2.Which type of food provider (restaurant, grocery store, etc.) contributes the most food?
query2="""SELECT Provider_Type, SUM(Quantity) AS Total_Quantity
          FROM FoodListings
          GROUP BY Provider_Type
          ORDER BY Total_Quantity DESC
          LIMIT 4;
       """
mycursor.execute(query2)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

      Provider_Type Total_Quantity
0        Restaurant           6923
1       Supermarket           6696
2  Catering Service           6116
3     Grocery Store           6059


In [40]:
#3.What is the contact information of food providers in a specific city?
query3="""select Name,Type,Address,Contact from Providers where city='New Jessica' """
mycursor.execute(query3)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

               Name         Type  \
0  Gonzales-Cochran  Supermarket   

                                             Address          Contact  
0  74347 Christopher Extensions\nAndreamouth, OK ...  +1-600-220-0480  


In [41]:
#4.Which receivers have claimed the most food?
query4="""SELECT 
          r.Name AS Receiver_Name,
          COUNT(c.Claim_ID) AS Total_Claims
          FROM Claims c
          JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID
          GROUP BY r.Name
          ORDER BY Total_Claims DESC
          LIMIT 10;
          """
mycursor.execute(query4)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

       Receiver_Name  Total_Claims
0  William Frederick             5
1       Matthew Webb             5
2       Scott Hunter             5
3     Anthony Garcia             5
4    Alexandra Owens             4
5         Betty Reid             4
6    Jennifer Nelson             4
7   Kristina Simpson             4
8    Kristine Martin             4
9         Alvin West             4


In [42]:
#5.What is the total quantity of food available from all providers?
query5="""SELECT SUM(Quantity) AS Total_Food_Quantity
        FROM FoodListings;
        """
mycursor.execute(query5)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

  Total_Food_Quantity
0               25794


In [43]:
#6.Which city has the highest number of food listings?
query6=""" SELECT Location AS City,
           COUNT(Food_ID) AS Total_Listings
           FROM FoodListings 
           GROUP BY Location
           ORDER BY Total_Listings DESC
           LIMIT 1;"""
mycursor.execute(query6)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

        City  Total_Listings
0  New Carol               6


In [44]:
#7.What are the most commonly available food types?
query7 = """
        SELECT Food_Type, COUNT(*) AS Count
        FROM FoodListings
        GROUP BY Food_Type
        ORDER BY Count DESC;
        """
mycursor.execute(query7)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

        Food_Type  Count
0      Vegetarian    336
1           Vegan    334
2  Non-Vegetarian    330


In [45]:
#8.How many food claims have been made for each food item?
query8=""" SELECT  f.Food_Name,
        COUNT(c.Claim_ID) AS Total_Claims
        FROM Claims c
        JOIN FoodListings f ON c.Food_ID = f.Food_ID
        GROUP BY f.Food_Name
        ORDER BY Total_Claims DESC;"""
mycursor.execute(query8)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

    Food_Name  Total_Claims
0        Rice           122
1        Soup           114
2       Dairy           110
3        Fish           108
4       Salad           106
5     Chicken           102
6       Bread            94
7       Pasta            87
8  Vegetables            86
9      Fruits            71


In [46]:
#9. Which provider has had the highest number of successful food claims?
query9="""  SELECT p.Name AS Provider_Name,
            COUNT(c.Claim_ID) AS Successful_Claims
            FROM Claims c
            JOIN FoodListings f ON c.Food_ID = f.Food_ID
            JOIN Providers p ON f.Provider_ID = p.Provider_ID
            WHERE c.Status = 'Completed'
            GROUP BY p.Name
            ORDER BY Successful_Claims DESC
            LIMIT 1;"""
mycursor.execute(query9)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

  Provider_Name  Successful_Claims
0   Barry Group                  5


In [47]:
#10.What percentage of food claims are completed vs. pending vs. canceled?
query10="""SELECT Status,
           COUNT(Claim_ID) AS Claim_Count,
           (COUNT(Claim_ID) * 100.0 / (SELECT COUNT(*) FROM Claims)) AS Percentage_of_Total
           FROM Claims
           GROUP BY Status
           ORDER BY Claim_Count DESC;"""
mycursor.execute(query10)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

      Status  Claim_Count Percentage_of_Total
0  Completed          339            33.90000
1  Cancelled          336            33.60000
2    Pending          325            32.50000


In [48]:
#11.What is the average quantity of food claimed per receiver?
query11="""  SELECT r.Name AS Receiver_Name,
             AVG(f.Quantity) AS Avg_Quantity_Claimed
             FROM Claims c
             JOIN FoodListings f ON c.Food_ID = f.Food_ID
             JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID
             GROUP BY r.Name
             ORDER BY Avg_Quantity_Claimed DESC;"""
mycursor.execute(query11)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

          Receiver_Name Avg_Quantity_Claimed
0           Nancy Jones              50.0000
1            Lisa Pitts              50.0000
2    Christopher Wright              50.0000
3           Nancy Silva              50.0000
4          Peggy Knight              50.0000
..                  ...                  ...
615       John Reynolds               2.0000
616     Matthew Johnson               2.0000
617      Kenneth Barnes               2.0000
618        Amanda Kline               1.0000
619       Joshua Hooper               1.0000

[620 rows x 2 columns]


In [49]:
#12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?
query12=""" SELECT f.Meal_Type,
            COUNT(c.Claim_ID) AS Total_Claims
            FROM Claims c
            JOIN FoodListings f ON c.Food_ID = f.Food_ID
            GROUP BY f.Meal_Type
            ORDER BY Total_Claims DESC;"""
mycursor.execute(query12)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

   Meal_Type  Total_Claims
0  Breakfast           278
1      Lunch           250
2     Snacks           240
3     Dinner           232


In [50]:
#13.What is the total quantity of food donated by each provider?
query13=""" SELECT 
            p.Name AS Provider_Name,
            SUM(f.Quantity) AS Total_Donated_Quantity
            FROM Providers p
            JOIN FoodListings f ON p.Provider_ID = f.Provider_ID
            GROUP BY p.Name
            ORDER BY Total_Donated_Quantity DESC;
        """
mycursor.execute(query13)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

                       Provider_Name Total_Donated_Quantity
0                         Miller Inc                    217
1                        Barry Group                    179
2         Evans, Wright and Mitchell                    158
3                        Smith Group                    150
4                       Campbell LLC                    145
..                               ...                    ...
623                   Reyes and Sons                      1
624                   Gonzales-Moore                      1
625                  Norris-Townsend                      1
626                        Mcgee PLC                      1
627  Martinez, Armstrong and Carroll                      1

[628 rows x 2 columns]


In [51]:
#14.Which city has the highest number of food providers?
query14="""SELECT City, COUNT(*) AS Total_Providers
FROM Providers
GROUP BY City
ORDER BY Total_Providers DESC
LIMIT 1;"""

mycursor.execute(query14)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

        City  Total_Providers
0  New Carol                3


In [52]:
#15.How many food items are near expiry (within next 7 days)?
query15="""SELECT Food_Name, Expiry_Date, DATEDIFF(Expiry_Date, CURDATE()) AS Days_Left
           FROM FoodListings
           WHERE Expiry_Date BETWEEN CURDATE() AND DATE_ADD(CURDATE(), INTERVAL 7 DAY);"""
mycursor.execute(query15)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

Empty DataFrame
Columns: [Food_Name, Expiry_Date, Days_Left]
Index: []


In [53]:
#16.Which city generates the most food claims?
query16="""SELECT 
    r.City,
    COUNT(c.Claim_ID) AS Total_Claims
FROM Claims c
JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.City
ORDER BY Total_Claims DESC;"""
mycursor.execute(query16)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

               City  Total_Claims
0        Smithshire             5
1        Emilymouth             5
2          Greenton             5
3      Port Richard             5
4         Port Dean             5
..              ...           ...
602    Matthewmouth             1
603  Port Kellifort             1
604     Andrewsport             1
605    Lake Jeffery             1
606   Shelbychester             1

[607 rows x 2 columns]


In [54]:
#17.How many claims are still pending per city?
query17="""SELECT 
    r.City,
    COUNT(c.Claim_ID) AS Pending_Claims
FROM Claims c
JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID
WHERE c.Status = 'Pending'
GROUP BY r.City
ORDER BY Pending_Claims DESC;"""
mycursor.execute(query17)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

                    City  Pending_Claims
0              Darinview               3
1           Spencermouth               3
2               New Tina               3
3      West Courtneyport               3
4             Lake Shawn               2
..                   ...             ...
271        North Michael               1
272  South Samanthaburgh               1
273        New Seanburgh               1
274         Brianchester               1
275          Travishaven               1

[276 rows x 2 columns]


In [55]:
#18.Monthly trend of food donations
query18="""SELECT DATE_FORMAT(Expiry_Date, '%Y-%m') AS Month,
           SUM(Quantity) AS Total_Donations
           FROM FoodListings
           GROUP BY Month
           ORDER BY Month;"""
mycursor.execute(query18)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

     Month Total_Donations
0  2025-03           25794


In [56]:
#19.Which receiver type has the most claims?
query19="""SELECT r.Type AS Receiver_Type, COUNT(c.Claim_ID) AS Total_Claims
FROM Claims c
JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.Type
ORDER BY Total_Claims DESC;"""
mycursor.execute(query19)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

  Receiver_Type  Total_Claims
0           NGO           272
1       Charity           268
2       Shelter           230
3    Individual           230


In [57]:
#20.Average number of food listings per provider
query20="""SELECT p.Name AS Provider_Name, COUNT(f.Food_ID) AS Total_Listings,
           AVG(f.Quantity) AS Avg_Quantity_Per_Listing
           FROM Providers p
           LEFT JOIN FoodListings f ON p.Provider_ID = f.Provider_ID
           GROUP BY p.Name
           ORDER BY Total_Listings DESC;"""
mycursor.execute(query20)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

                    Provider_Name  Total_Listings Avg_Quantity_Per_Listing
0                      Miller Inc               8                  27.1250
1                    Campbell LLC               7                  20.7143
2                     Barry Group               6                  29.8333
3                     Smith Group               5                  30.0000
4                     Daniels-Kim               5                  18.0000
..                            ...             ...                      ...
968  Morris, Serrano and Roberson               0                     None
969                   Spencer LLC               0                     None
970                Miller-Sanders               0                     None
971    Porter, Hernandez and Pace               0                     None
972     Vasquez, Ruiz and Flowers               0                     None

[973 rows x 3 columns]


In [58]:
#21.Do cities with more providers have fewer pending claims?
query21="""SELECT 
    p.City,
    COUNT(DISTINCT p.Provider_ID) AS Providers,
    SUM(CASE WHEN c.Status = 'Pending' THEN 1 ELSE 0 END) AS Pending_Claims
FROM Providers p
LEFT JOIN FoodListings f ON p.Provider_ID = f.Provider_ID
LEFT JOIN Claims c ON f.Food_ID = c.Food_ID
GROUP BY p.City
ORDER BY Providers DESC;"""
mycursor.execute(query21)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

                         City  Providers Pending_Claims
0                   New Carol          3              1
1    South Christopherborough          3              0
2                 Bradleyport          2              2
3                  Davidville          2              1
4                East Anthony          2              1
..                        ...        ...            ...
958               Wrightville          1              0
959                 Yatesside          1              1
960              Youngchester          1              1
961              Zimmermanton          1              0
962            Zimmermanville          1              0

[963 rows x 3 columns]


In [59]:
#22.Which cities have the highest number of completed food claims?
query22="""SELECT f.Location AS City, COUNT(c.Claim_ID) AS Completed_Claims
FROM Claims c
JOIN FoodListings f ON c.Food_ID = f.Food_ID
WHERE c.Status = 'Completed'
GROUP BY f.Location
ORDER BY Completed_Claims DESC;"""
mycursor.execute(query22)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

                   City  Completed_Claims
0         South Kathryn                 5
1             New Carol                 4
2            Devinmouth                 4
3             Coleburgh                 4
4      East Heatherport                 4
..                  ...               ...
243  North Edwinchester                 1
244       North Crystal                 1
245        Lake Heather                 1
246        Harrisonbury                 1
247          Port Karen                 1

[248 rows x 2 columns]


In [60]:
#23.Which food type has the highest average quantity donated per listing?
query23="""SELECT Food_Type,
       ROUND(AVG(Quantity), 2) AS Avg_Quantity_Donated
FROM FoodListings
GROUP BY Food_Type
ORDER BY Avg_Quantity_Donated DESC;"""
mycursor.execute(query23)
rows = mycursor.fetchall()
columns = [desc[0] for desc in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
print(df)

        Food_Type Avg_Quantity_Donated
0           Vegan                26.34
1  Non-Vegetarian                26.23
2      Vegetarian                24.82
